## Model training and tuning for the materials sector

In [1]:
import pandas as pd
import numpy as np

import time

from darts import TimeSeries, concatenate
from darts.utils.model_selection import train_test_split
from darts.models.forecasting.xgboost import XGBModel
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mape, mae
from darts.models.forecasting.prophet_model import Prophet
from darts.models.forecasting.sf_auto_theta import AutoTheta
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from darts.models.forecasting.sf_auto_arima import AutoARIMA
from darts.dataprocessing.transformers.boxcox import BoxCox

Importing plotly failed. Interactive plots will not work.
d:\investing-bot\.venv\Lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])

df_materials = df[df['GICS Sector'] == 'Materials'].sort_values(by=['ticker', 'date']).reset_index(drop=True)

In [3]:
df['GICS Sector'].value_counts()

GICS Sector
Industrials               193459
Financials                190485
Information Technology    177149
Health Care               151348
Consumer Discretionary    130185
Consumer Staples           96649
Real Estate                79757
Utilities                  78786
Materials                  63394
Energy                     58523
Communication Services     58017
Name: count, dtype: int64

In [4]:
df_materials['ticker'].unique()

array(['ALB', 'AMCR', 'APD', 'AVY', 'BALL', 'CF', 'CTVA', 'DD', 'DOW',
       'ECL', 'EMN', 'FCX', 'IFF', 'IP', 'LIN', 'LYB', 'MLM', 'MOS',
       'NEM', 'NUE', 'PKG', 'PPG', 'SHW', 'STLD', 'SW', 'VMC'],
      dtype=object)

In [5]:
data = {}
static_covs = pd.DataFrame({'ticker': df_materials['ticker'].unique()})

for ticker, group in df_materials.groupby('ticker'):
    series = TimeSeries.from_dataframe(
        group,
        time_col='date',
        value_cols=['close', 'high', 'low', 'open', 'volume'],
        freq='B',
        static_covariates=pd.Series(ticker)
    )
    data[ticker] = series

In [6]:
# encode each ticker as a static covariate
transformer = StaticCovariatesTransformer()
ts = [d for d in data.values()]
transformer.fit(ts)

for ticker, series in data.items():
    data[ticker] = transformer.transform(series)

In [7]:
train = {}
test = {}

for ticker, series in data.items():
    train[ticker], test[ticker] = train_test_split(series, test_size=0.1)
    last_train = train[ticker][-1:]
    test_with_context = last_train.append(test[ticker]) # append last point of train to test as our forecast requires it
    test[ticker] = test_with_context

## Gradient Boosting

### Train a local model first

In [8]:
models = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = XGBModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models[ticker] = model

Training model for ALB
Model for ALB trained in 0.35 seconds
**************************************************
Training model for AMCR
Model for AMCR trained in 0.33 seconds
**************************************************
Training model for APD
Model for APD trained in 0.39 seconds
**************************************************
Training model for AVY
Model for AVY trained in 0.33 seconds
**************************************************
Training model for BALL
Model for BALL trained in 0.38 seconds
**************************************************
Training model for CF
Model for CF trained in 0.37 seconds
**************************************************
Training model for CTVA
Model for CTVA trained in 0.33 seconds
**************************************************
Training model for DD
Model for DD trained in 0.37 seconds
**************************************************
Training model for DOW
Model for DOW trained in 0.30 seconds
*****************************************

In [9]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes[ticker] = mape_
    maes[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ALB, MAPE: 8.85, MAE: 7.67
**************************************************
Ticker: AMCR, MAPE: 3.70, MAE: 0.38
**************************************************
Ticker: APD, MAPE: 13.27, MAE: 39.96
**************************************************
Ticker: AVY, MAPE: 2.57, MAE: 5.16
**************************************************
Ticker: BALL, MAPE: 3.00, MAE: 1.72
**************************************************
Ticker: CF, MAPE: 4.34, MAE: 3.57
**************************************************
Ticker: CTVA, MAPE: 4.27, MAE: 2.62
**************************************************
Ticker: DD, MAPE: 6.58, MAE: 5.08
**************************************************
Ticker: DOW, MAPE: 4.76, MAE: 1.83
**************************************************
Ticker: ECL, MAPE: 7.96, MAE: 19.80
**************************************************
Ticker: EMN, MAPE: 3.60, MAE: 3.52
**************************************************
Ticker: FCX, MAPE: 6.32, MAE: 2.73
***************

In [10]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(6.318322860605256, 10.366251712857887)

### Global model

In [11]:
print(f"Training global model")
start = time.time()
model = XGBModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    },
    use_static_covariates=True,
    n_estimators=1000,
    eta=0.01,
    max_depth=8,
    subsample=0.8
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 11.22 seconds
**************************************************


In [12]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes[ticker] = mape_
    maes[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for ALB, MAPE: 9.12, MAE: 7.21
**************************************************
Global model for AMCR, MAPE: 1.92, MAE: 0.19
**************************************************
Global model for APD, MAPE: 2.70, MAE: 7.83
**************************************************
Global model for AVY, MAPE: 1.98, MAE: 3.96
**************************************************
Global model for BALL, MAPE: 8.78, MAE: 5.00
**************************************************
Global model for CF, MAPE: 5.63, MAE: 4.44
**************************************************
Global model for CTVA, MAPE: 2.92, MAE: 1.73
**************************************************
Global model for DD, MAPE: 3.13, MAE: 2.25
**************************************************
Global model for DOW, MAPE: 8.02, MAE: 2.88
**************************************************
Global model for ECL, MAPE: 1.93, MAE: 4.72
**************************************************
Global model for EMN, MAPE: 4.24, MAE: 3.78
*****

In [13]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(4.9154103202791415, 7.912543004498281)

## Prophet model

In [14]:
models_prophet = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = Prophet(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        seasonality_mode='multiplicative',
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_prophet[ticker] = model

Training model for ALB


22:08:43 - cmdstanpy - INFO - Chain [1] start processing
22:08:45 - cmdstanpy - INFO - Chain [1] done processing


Model for ALB trained in 2.63 seconds
**************************************************
Training model for AMCR


22:08:46 - cmdstanpy - INFO - Chain [1] start processing
22:08:48 - cmdstanpy - INFO - Chain [1] done processing


Model for AMCR trained in 2.93 seconds
**************************************************
Training model for APD


...

Model for STLD trained in 2.22 seconds
**************************************************
Training model for SW


22:09:51 - cmdstanpy - INFO - Chain [1] done processing


Model for SW trained in 0.24 seconds
**************************************************
Training model for VMC


22:09:52 - cmdstanpy - INFO - Chain [1] start processing
22:09:54 - cmdstanpy - INFO - Chain [1] done processing


Model for VMC trained in 2.66 seconds
**************************************************


In [15]:
mapes_prophet = {}
maes_prophet = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_prophet[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_prophet[ticker] = mape_
    maes_prophet[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ALB, MAPE: 48.40, MAE: 40.74
**************************************************
Ticker: AMCR, MAPE: 27.02, MAE: 2.72
**************************************************
Ticker: APD, MAPE: 55.24, MAE: 161.97
**************************************************
Ticker: AVY, MAPE: 12.17, MAE: 23.74
**************************************************
Ticker: BALL, MAPE: 12.21, MAE: 7.25
**************************************************
Ticker: CF, MAPE: 18.13, MAE: 14.83
**************************************************
Ticker: CTVA, MAPE: 9.79, MAE: 5.92
**************************************************
Ticker: DD, MAPE: 23.36, MAE: 18.46
**************************************************
Ticker: DOW, MAPE: 35.79, MAE: 12.63
**************************************************
Ticker: ECL, MAPE: 6.92, MAE: 17.07
**************************************************
Ticker: EMN, MAPE: 15.29, MAE: 15.01
**************************************************
Ticker: FCX, MAPE: 22.79, MAE: 10.1

In [16]:
np.mean(list(mapes_prophet.values())), np.mean(list(maes_prophet.values()))

(21.150606005538062, 27.39301378933326)

## Theta model

In [17]:
models_theta = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoTheta(
        season_length=30
    )

    model.fit(
        series=train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_theta[ticker] = model

Training model for ALB
Model for ALB trained in 0.59 seconds
**************************************************
Training model for AMCR
Model for AMCR trained in 0.63 seconds
**************************************************
Training model for APD
Model for APD trained in 0.72 seconds
**************************************************
Training model for AVY
Model for AVY trained in 0.82 seconds
**************************************************
Training model for BALL
Model for BALL trained in 0.60 seconds
**************************************************
Training model for CF
Model for CF trained in 0.64 seconds
**************************************************
Training model for CTVA
Model for CTVA trained in 0.27 seconds
**************************************************
Training model for DD
Model for DD trained in 0.59 seconds
**************************************************
Training model for DOW
Model for DOW trained in 0.28 seconds
*****************************************

In [18]:
mapes_theta = {}
maes_theta = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_theta[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_theta[ticker] = mape_
    maes_theta[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ALB, MAPE: 47.39, MAE: 37.60
**************************************************
Ticker: AMCR, MAPE: 5.22, MAE: 0.54
**************************************************
Ticker: APD, MAPE: 14.33, MAE: 42.84
**************************************************
Ticker: AVY, MAPE: 13.79, MAE: 25.65
**************************************************
Ticker: BALL, MAPE: 18.04, MAE: 9.78
**************************************************
Ticker: CF, MAPE: 7.86, MAE: 6.63
**************************************************
Ticker: CTVA, MAPE: 3.29, MAE: 1.98
**************************************************
Ticker: DD, MAPE: 5.86, MAE: 4.38
**************************************************
Ticker: DOW, MAPE: 31.27, MAE: 11.10
**************************************************
Ticker: ECL, MAPE: 6.91, MAE: 17.23
**************************************************
Ticker: EMN, MAPE: 6.79, MAE: 6.27
**************************************************
Ticker: FCX, MAPE: 17.90, MAE: 6.93
*******

In [19]:
np.mean(list(mapes_theta.values())), np.mean(list(maes_theta.values()))

(13.338419316203742, 16.929362164119542)

## Linear Regression

### Local model

In [20]:
models_lr = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = LinearRegressionModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_lr[ticker] = model

Training model for ALB
Model for ALB trained in 0.13 seconds
**************************************************
Training model for AMCR
Model for AMCR trained in 0.13 seconds
**************************************************
Training model for APD
Model for APD trained in 0.12 seconds
**************************************************
Training model for AVY
Model for AVY trained in 0.12 seconds
**************************************************
Training model for BALL
Model for BALL trained in 0.13 seconds
**************************************************
Training model for CF
Model for CF trained in 0.12 seconds
**************************************************
Training model for CTVA
Model for CTVA trained in 0.09 seconds
**************************************************
Training model for DD
Model for DD trained in 0.12 seconds
**************************************************
Training model for DOW
Model for DOW trained in 0.09 seconds
*****************************************

In [21]:
mapes_lr = {}
maes_lr = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_lr[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_lr[ticker] = mape_
    maes_lr[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ALB, MAPE: 18.85, MAE: 15.04
**************************************************
Ticker: AMCR, MAPE: 1.90, MAE: 0.19
**************************************************
Ticker: APD, MAPE: 2.53, MAE: 7.32
**************************************************
Ticker: AVY, MAPE: 2.00, MAE: 3.91
**************************************************
Ticker: BALL, MAPE: 2.34, MAE: 1.33
**************************************************
Ticker: CF, MAPE: 3.17, MAE: 2.55
**************************************************
Ticker: CTVA, MAPE: 2.01, MAE: 1.20
**************************************************
Ticker: DD, MAPE: 7.08, MAE: 5.50
**************************************************
Ticker: DOW, MAPE: 13.26, MAE: 4.84
**************************************************
Ticker: ECL, MAPE: 1.95, MAE: 4.79
**************************************************
Ticker: EMN, MAPE: 4.28, MAE: 3.93
**************************************************
Ticker: FCX, MAPE: 4.92, MAE: 2.01
***************

In [22]:
np.mean(list(mapes_lr.values())), np.mean(list(maes_lr.values()))

(6018754202.666317, 6961394937.228497)

### Global model

In [23]:
print(f"Training global model")
start = time.time()
model = LinearRegressionModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    }
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 3.11 seconds
**************************************************


In [24]:
mapes_lr_global = {}
maes_lr_global = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes_lr_global[ticker] = mape_
    maes_lr_global[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for ALB, MAPE: 6.89, MAE: 5.76
**************************************************
Global model for AMCR, MAPE: 4.04, MAE: 0.40
**************************************************
Global model for APD, MAPE: 3.28, MAE: 9.47
**************************************************
Global model for AVY, MAPE: 2.14, MAE: 4.21
**************************************************
Global model for BALL, MAPE: 3.05, MAE: 1.74
**************************************************
Global model for CF, MAPE: 3.30, MAE: 2.64
**************************************************
Global model for CTVA, MAPE: 2.76, MAE: 1.65
**************************************************
Global model for DD, MAPE: 2.88, MAE: 2.11
**************************************************
Global model for DOW, MAPE: 5.17, MAE: 1.96
**************************************************
Global model for ECL, MAPE: 2.20, MAE: 5.37
**************************************************
Global model for EMN, MAPE: 3.52, MAE: 3.25
*****

In [25]:
np.mean(list(mapes_lr_global.values())), np.mean(list(maes_lr_global.values()))

(3.4718968819317078, 4.335975731234646)

## ARIMA

In [26]:
models_arima = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoARIMA()

    model.fit(
        series=train[ticker]['close'],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_arima[ticker] = model

Training model for ALB
Model for ALB trained in 0.53 seconds
**************************************************
Training model for AMCR
Model for AMCR trained in 1.81 seconds
**************************************************
Training model for APD
Model for APD trained in 0.45 seconds
**************************************************
Training model for AVY
Model for AVY trained in 1.48 seconds
**************************************************
Training model for BALL
Model for BALL trained in 3.64 seconds
**************************************************
Training model for CF
Model for CF trained in 0.58 seconds
**************************************************
Training model for CTVA
Model for CTVA trained in 0.14 seconds
**************************************************
Training model for DD
Model for DD trained in 0.21 seconds
**************************************************
Training model for DOW
Model for DOW trained in 0.16 seconds
*****************************************

In [ ]:
mapes_arima = {}
maes_arima = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_arima[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_arima[ticker] = mape_
    maes_arima[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ALB, MAPE: 42.09, MAE: 33.37
**************************************************
Ticker: AMCR, MAPE: 7.75, MAE: 0.80
**************************************************
Ticker: APD, MAPE: 16.52, MAE: 49.22
**************************************************
Ticker: AVY, MAPE: 13.99, MAE: 26.02
**************************************************
Ticker: BALL, MAPE: 15.30, MAE: 8.27
**************************************************
Ticker: CF, MAPE: 9.96, MAE: 8.40
**************************************************
Ticker: CTVA, MAPE: 4.35, MAE: 2.67
**************************************************
Ticker: DD, MAPE: 6.05, MAE: 4.54
**************************************************
Ticker: DOW, MAPE: 29.77, MAE: 10.57
**************************************************
Ticker: ECL, MAPE: 7.99, MAE: 19.90
**************************************************
Ticker: EMN, MAPE: 6.80, MAE: 6.43
**************************************************
Ticker: FCX, MAPE: 16.11, MAE: 6.25
*******

In [28]:
np.mean(list(mapes_arima.values())), np.mean(list(maes_arima.values()))

(13.231832505432386, 17.446585824120774)